In [107]:
# encoding='utf-8'
import json
import time
import numpy as np
import pandas as pd
import requests
from threading import Thread
from Cover_strategy import cancel_order
from loggerConfig import logger
from tools.Kline_analyze import get_all_klinedata,get_klinedata
from tools.Config import pricelimit, Trade_url, Queryorder_url, Fee, Query_tradeprice_url, updateCover_url
from tools.databasePool import r2, POOL, r5
from tools.get_market_info import get_currentprice1

In [75]:
platform='huobi'
symbol='eth_usdt'
granularity=3600
klinedata = get_klinedata(platform, symbol, granularity)
print(klinedata)

[['2021-05-06 04:00:00', 3479.71, 3484.41, 3434.61, 3459.53, 18413.438947098963], ['2021-05-06 05:00:00', 3458.55, 3465.94, 3420.0, 3426.85, 7034.647184193287], ['2021-05-06 06:00:00', 3426.73, 3536.16, 3426.73, 3521.55, 37877.591530958234], ['2021-05-06 07:00:00', 3520.91, 3548.0, 3494.4, 3524.01, 23965.02432298901], ['2021-05-06 08:00:00', 3524.01, 3539.08, 3448.1, 3475.93, 33535.36293055338], ['2021-05-06 09:00:00', 3474.96, 3494.83, 3440.3, 3474.82, 17221.886805747203], ['2021-05-06 10:00:00', 3474.3, 3504.88, 3461.09, 3483.65, 13733.09417351984], ['2021-05-06 11:00:00', 3483.64, 3483.65, 3447.35, 3469.04, 13433.96348017623], ['2021-05-06 12:00:00', 3469.04, 3501.19, 3432.0, 3440.47, 29311.19171773106], ['2021-05-06 13:00:00', 3440.46, 3463.94, 3425.1, 3450.46, 12304.901963680906], ['2021-05-06 14:00:00', 3450.45, 3450.45, 3380.85, 3392.45, 22323.831450513564], ['2021-05-06 15:00:00', 3392.86, 3444.67, 3375.5, 3438.98, 17643.815170034322], ['2021-05-06 16:00:00', 3438.99, 3456.65, 

In [76]:
# 简单移动平均线
def MA(klinedata):
    dataframe = dict({'time': [i[0] for i in klinedata],
                      'low':[float(i[3]) for i in klinedata],
                      "close": [float(i[4]) for i in klinedata],
                      "volumn": [float(i[5]) for i in klinedata]})
    data = pd.DataFrame(dataframe)
    data['MA5'] = data['close'].rolling(5).mean()
    data['MA30'] = data['close'].rolling(30).mean()
    data['MA_sign'] = 0
    MA_position = data['MA5'] > data['MA30']
    # MA_position=data['MA20']>data['MA30']
    data.loc[MA_position[(MA_position == True) & (MA_position.shift() == False)].index, 'MA_sign'] = 1  # 金叉
    data.loc[MA_position[(MA_position == False) & (MA_position.shift() == True)].index, 'MA_sign'] = 2  # 死叉
    # sign = data.loc[199, "MA_sign"]
    # print(data[150:])
    # print(sign)
    return data

In [81]:
MA_data = MA(klinedata)
ma_sign_array = MA_data['MA_sign'].values.tolist()
low_array = MA_data['low'].values.tolist()
volumn_array = MA_data['volumn'].values.tolist()

In [82]:
ma_sign_array[::-1].index(1)

58

In [83]:
print(ma_sign_array[0:-58])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [84]:
ma_sign_array=ma_sign_array[0:-58]
low_array=low_array[0:-58]
volumn_array=volumn_array[0:-58]

In [85]:
print(ma_sign_array)
print("+++++++++++++++++++++++++")
print(low_array)
print("+++++++++++++++++++++++++++")
print(volumn_array)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
+++++++++++++++++++++++++
[3434.61, 3420.0, 3426.73, 3494.4, 3448.1, 3440.3, 3461.09, 3447.35, 3432.0, 3425.1, 3380.85, 3375.5, 3422.71, 3418.27, 3440.85, 3467.89, 3476.7, 3460.0, 3459.7, 3507.02, 3528.58, 3537.92, 3400.01, 3407.08, 3435.22, 3458.0, 3502.15, 3450.72, 3464.27, 3448.03, 3402.16, 3398.39, 3354.05, 3397.95, 3397.66, 3422.16, 3408.45, 3430.74, 3448.0, 3426.0, 3422.07, 3479.0, 3472.0, 3495.48, 3530.79, 3521.7, 3513.58, 3498.58, 3477.18, 3456.0, 3421.0, 3438.1, 3449.11, 3516.59, 3510.81, 3522.55, 3523.3, 3528.56, 3508.12, 3507.16, 3526.6, 3535.91, 3521.11, 

In [86]:
dead_cross_position = ma_sign_array[::-1].index(2)

In [87]:
dead_cross_position

19

In [88]:
ma_sign_array_1=ma_sign_array[-(dead_cross_position + 1):]

In [89]:
ma_sign_array_1

[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [90]:
low_array_1 = low_array[-(dead_cross_position + 1):]
volumn_array_1 = volumn_array[-(dead_cross_position + 1):]

In [91]:
print(ma_sign_array_1)
print('++++++++++++++++++++++++++')
print(low_array_1)
print('++++++++++++++++++++++++++')
print(volumn_array_1)

[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
++++++++++++++++++++++++++
[3975.56, 3883.44, 3855.19, 3827.0, 3811.0, 3774.25, 3833.52, 3844.03, 3871.65, 3917.66, 3903.0, 3918.34, 4002.49, 3962.33, 3906.0, 3896.01, 3944.22, 3962.07, 3989.68, 3976.53]
++++++++++++++++++++++++++
[31400.595835576452, 37161.83221190951, 47466.10822479746, 41100.29186636757, 28475.29205169473, 38331.96733554354, 21881.455915694605, 22526.954752589176, 17088.505937258808, 18712.013557863545, 19454.078983655836, 22788.300540576223, 26623.875443882447, 20292.052477754005, 31228.826308353462, 22603.08462525154, 15848.10152617049, 15669.681934229864, 49317.42875456034, 13493.862784150513]


In [92]:
low_min_position = low_array_1.index(min(low_array_1))

In [93]:
low_min_position

5

In [94]:
volumn_mean = np.mean(volumn_array[:low_min_position])

In [95]:
volumn_array[low_min_position]

17221.886805747203

In [97]:
volumn_mean*2.5

60413.032457896436

In [98]:
volumn_array[low_min_position]>volumn_mean*2.5

False

In [99]:
from tools.Config import symbols

In [190]:
symbols=['btc_usdt',
'eth_usdt',
'ht_usdt',
'dot_usdt',
'xrp_usdt',
'link_usdt',
'bch_usdt',
'ltc_usdt',
'bsv_usdt',
'ada_usdt',
'eos_usdt',
'trx_usdt',
'xmr_usdt',
'iota_usdt',
'dash_usdt',
'etc_usdt',
'zec_usdt',
'usdc_usdt',
'pax_usdt',
'wbtc_usdt',
'btc3l_usdt',
'btc3s_usdt',
'eth3l_usdt',
'eth3s_usdt',
'link3l_usdt',
'link3s_usdt',
'shib_usdt',
'doge_usdt',
'cspr_usdt',
'fil_usdt',
'icp_usdt',
'btt_usdt',
'lat_usdt',
'uni_usdt',
'zrx_usdt',
'zks_usdt',
'zil_usdt',
'zen_usdt',
'zec3l_usdt',
'zec3s_usdt',
'yfii_usdt',
'yfi_usdt',
'yee_usdt',
'yam_usdt',
'xtz_usdt',
'xrt_usdt',
'xrp3l_usdt',
'xrp3s_usdt',
'xmx_usdt',
'xlm_usdt',
'xem_usdt',
'xch_usdt',
'wxt_usdt',
'wtc_usdt',
'woo_usdt',
'wnxm_usdt',
'wicc_usdt',
'waxp_usdt',
'waves_usdt',
'vsys_usdt',
'vidy_usdt',
'vet_usdt',
'value_usdt',
'uuu_usdt',
'utk_usdt',
'uni2l_usdt',
'uni2s_usdt',
'uma_usdt',
'uip_usdt',
'tt_usdt',
'btc1s_usdt',
'bsv3l_usdt',
'bsv3s_usdt',
'bor_usdt',
'bnt_usdt',
'blz_usdt',
'bix_usdt',
'bhd_usdt',
'beth_usdt',
'bcha_usdt',
'bch3l_usdt',
'bch3s_usdt',
'bat_usdt',
'band_usdt',
'bal_usdt',
'bags_usdt',
'badger_usdt',
'axs_usdt',
'avax_usdt',
'auction_usdt',
'atp_usdt',
'atom_usdt',
'ast_usdt',
'arpa_usdt',
'ar_usdt',
'apn_usdt',
'api3l_usdt',
'ant_usdt',
'ankr_usdt',
'algo_usdt',
'akro_usdt',
'ae_usdt',
'act_usdt',
'ach_usdt',
'abt_usdt',
'aave_usdt',
'aac_usdt',
'1inch_usdt',]


In [191]:
# 获取k线数据（现货）
def get_klinedata(platform, symbol, granularity):
    klinedata = []
    try:
        # [时间，开盘价，最高价，最低价，收盘价，交易量]
        if platform == "okex":  # 200条数据，时间粒度60,300,3600,86400……
            res = requests.get("https://www.okex.com/api/spot/v3/instruments/{}/candles?granularity={}".format(
                symbol.upper().replace("_", "-"), granularity), timeout=1)
            klinedata = json.loads(res.content.decode())[::-1]  # 时间升序排列
            for i in klinedata:
                t = i[0].replace("T", " ").replace(".000Z", "")
                timeStruct = time.strptime(t, "%Y-%m-%d %H:%M:%S")
                timeStamp = int(time.mktime(timeStruct)) + 60 * 60 * 8
                i[0] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(timeStamp))
        if platform == "huobi":  # 200条数据，时间粒度1min, 5min, 15min, 30min, 60min, 4hour, 1day, 1mon, 1week, 1year
            huobi_granularity_dict = {60: "1min", 300: "5min", 900: "15min", 1800: "30min", 3600: "60min",
                                      14400: "4hour", 86400: "1day", 604800: "1week", 2592000: "mon",
                                      946080000: "1year"}
            res = requests.get("https://api.huobi.pro/market/history/kline?period={}&size=200&symbol={}".format(
                huobi_granularity_dict[granularity], symbol.replace("_", "")), timeout=1)
            data = json.loads(res.content.decode())["data"]
            klinedata = []
            for i in data:
                l = [
                    time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(i["id"])),
                    i["open"],
                    i["high"],
                    i["low"],
                    i["close"],
                    i["amount"]
                ]
                klinedata.append(l)
            klinedata = klinedata[::-1]
        if platform == "binance":  # 200条数据，时间粒度1m, 5m, 15m, 30m, 1h, 4h, 1d
            binance_granularity_dict = {60: "1m", 300: "5m", 900: "15m", 1800: "30m", 3600: "1h",
                                        14400: "4h", 86400: "1d"}
            res = requests.get("https://www.binancezh.cc/api/v3/klines?symbol={}&interval={}&limit=200".format(
                symbol.upper().replace("_", ""), binance_granularity_dict[granularity]), timeout=1)
            data = json.loads(res.content.decode())
            klinedata = []
            for i in data:
                l = [
                    time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(int(str(i[0])[:10]))),
                    i[1],
                    i[2],
                    i[3],
                    i[4],
                    i[5],
                ]
                klinedata.append(l)
    except:
        pass
    finally:
        symbol_klinedata=(symbol,klinedata)
    return symbol_klinedata

In [192]:
def get_all_symbols_klinedata(platform):
    result_list = []
    thread_list = []
    for symbol in symbols:
        t = MyThread(get_klinedata, (platform, symbol, 3600,))
        thread_list.append(t)
        t.start()
    for t in thread_list:
        t.join()
        result_list.append(t.get_result())
    return result_list

In [193]:
class MyThread(Thread):
    def __init__(self, func, args):
        super(MyThread, self).__init__()
        self.func = func
        self.args = args

    def run(self):
        self.result = self.func(*self.args)

    def get_result(self):
        try:
            return self.result
        except Exception:
            return None

In [ ]:
def gold_cross(symbol,klinedata):
    MA_data = MA(klinedata)
    ma_sign_array = MA_data['MA_sign'].values.tolist()
    low_array = MA_data['low'].values.tolist()
    volumn_array = MA_data['volumn'].values.tolist()
    flag = 0
    try:
        if ma_sign_array[-1] == 1:
            dead_cross_position = ma_sign_array[::-1].index(2)
            low_array = low_array[-(dead_cross_position + 1):]
            volumn_array = volumn_array[-(dead_cross_position + 1):]
            low_min_position = low_array.index(min(low_array))
            volumn_mean = np.mean(volumn_array[:low_min_position])
            if volumn_array[low_min_position] >= volumn_mean * 2.5:
                flag = 1
    except Exception as e:
        pass
    finally:
        return flag

In [194]:
symbols_klinedata=get_all_symbols_klinedata('huobi')

In [195]:
#symbols_klinedata=get_all_symbols_klinedata('huobi')
symbols_klinedata

[('btc_usdt',
  [['2021-05-09 09:00:00',
    58518.92,
    59200.0,
    58335.84,
    59100.0,
    1512.035960997806],
   ['2021-05-09 10:00:00',
    59099.99,
    59112.7,
    58479.67,
    58583.05,
    1325.2549407673591],
   ['2021-05-09 11:00:00',
    58583.05,
    58995.09,
    58500.0,
    58619.44,
    2156.489413917564],
   ['2021-05-09 12:00:00',
    58619.44,
    58850.0,
    57806.2,
    58152.68,
    4191.076104020933],
   ['2021-05-09 13:00:00',
    58152.68,
    58559.98,
    57936.52,
    58438.28,
    995.9608390496606],
   ['2021-05-09 14:00:00',
    58438.22,
    58568.95,
    58234.69,
    58339.8,
    744.6599631483689],
   ['2021-05-09 15:00:00',
    58342.69,
    58349.25,
    57700.0,
    57956.33,
    1303.8744678514197],
   ['2021-05-09 16:00:00',
    57956.33,
    58251.24,
    57815.29,
    58152.41,
    795.0229972456904],
   ['2021-05-09 17:00:00',
    58158.16,
    58388.0,
    57582.12,
    57956.04,
    1172.6108200882293],
   ['2021-05-09 18:00:00',
  

In [196]:
for i in symbols_klinedata:
    symbol=i[0]
    klinedata=i[1]
    if klinedata:
        print(symbol)
    #print(symbol,gold_cross(symbol,klinedata))

btc_usdt
eth_usdt
ht_usdt
xrp_usdt
link_usdt
ltc_usdt


In [198]:
path=r'C:\Users\Administrator\Desktop\新泽熙策略思路摸索.xlsx'
import pandas as pd
df=pd.read_excel(path,sheet_name="Sheet3")

In [200]:
symbols=df['币种'].values.tolist()

In [202]:
print(symbols)

['btc_usdt', 'eth_usdt', 'ht_usdt', 'dot_usdt', 'xrp_usdt', 'link_usdt', 'bch_usdt', 'ltc_usdt', 'bsv_usdt', 'ada_usdt', 'eos_usdt', 'trx_usdt', 'xmr_usdt', 'iota_usdt', 'dash_usdt', 'etc_usdt', 'zec_usdt', 'usdc_usdt', 'pax_usdt', 'wbtc_usdt', 'btc3l_usdt', 'btc3s_usdt', 'eth3l_usdt', 'eth3s_usdt', 'link3l_usdt', 'link3s_usdt', 'shib_usdt', 'doge_usdt', 'cspr_usdt', 'fil_usdt', 'icp_usdt', 'btt_usdt', 'lat_usdt', 'uni_usdt', 'nano_usdt', 'sushi_usdt', 'mdx_usdt', 'xch_usdt', 'chz_usdt', 'ach_usdt', 'mx_usdt', 'crv_usdt', 'soc_usdt', 'matic_usdt', 'zks_usdt', 'kan_usdt', 'xlm_usdt', 'aave_usdt', 'ont_usdt', 'atom_usdt', 'yee_usdt', 'egt_usdt', 'omg_usdt', 'cnns_usdt', 'qtum_usdt', 'theta_usdt', 'ckb_usdt', 'btm_usdt', 'iotx_usdt', 'hit_usdt', 'lol_usdt', 'iost_usdt', 'vet_usdt', 'ksm_usdt', 'act_usdt', 'tnb_usdt', 'nest_usdt', 'pai_usdt', 'ar_usdt', 'skm_usdt', 'gt_usdt', 'mana_usdt', 'yfi_usdt', 'chr_usdt', 'dta_usdt', 'neo_usdt', 'jst_usdt', 'xem_usdt', 'lrc_usdt', '1inch_usdt', 'flo

In [204]:
symbols = ['btc_usdt', 'eth_usdt', 'ht_usdt', 'dot_usdt', 'xrp_usdt', 'link_usdt', 'bch_usdt', 'ltc_usdt',
               'bsv_usdt', 'ada_usdt', 'eos_usdt', 'trx_usdt', 'xmr_usdt', 'iota_usdt', 'dash_usdt', 'etc_usdt',
               'zec_usdt', 'usdc_usdt', 'pax_usdt', 'wbtc_usdt', 'btc3l_usdt', 'btc3s_usdt', 'eth3l_usdt', 'eth3s_usdt',
               'link3l_usdt', 'link3s_usdt', 'shib_usdt', 'doge_usdt', 'cspr_usdt', 'fil_usdt', 'icp_usdt', 'btt_usdt',
               'lat_usdt', 'uni_usdt', 'nano_usdt', 'sushi_usdt', 'mdx_usdt', 'xch_usdt', 'chz_usdt', 'ach_usdt',
               'mx_usdt', 'crv_usdt', 'soc_usdt', 'matic_usdt', 'zks_usdt', 'kan_usdt', 'xlm_usdt', 'aave_usdt',
               'ont_usdt', 'atom_usdt', 'yee_usdt', 'egt_usdt', 'omg_usdt', 'cnns_usdt', 'qtum_usdt', 'theta_usdt',
               'ckb_usdt', 'btm_usdt', 'iotx_usdt', 'hit_usdt', 'lol_usdt', 'iost_usdt', 'vet_usdt', 'ksm_usdt',
               'act_usdt', 'tnb_usdt', 'nest_usdt', 'pai_usdt', 'ar_usdt', 'skm_usdt', 'gt_usdt', 'mana_usdt',
               'yfi_usdt', 'chr_usdt', 'dta_usdt', 'neo_usdt', 'jst_usdt', 'xem_usdt', 'lrc_usdt', '1inch_usdt',
               'flow_usdt', 'zen_usdt', 'mask_usdt', 'sol_usdt', 'hpt_usdt', 'bat_usdt', 'near_usdt', 'gxc_usdt',
               'enj_usdt', 'itc_usdt', 'bix_usdt', 'sc_usdt', 'mass_usdt', 'new_usdt', 'fsn_usdt', 'algo_usdt',
               'wxt_usdt', 'grt_usdt', 'xtz_usdt', 'snx_usdt', 'ring_usdt', 'yfii_usdt', 'hbar_usdt', 'wicc_usdt',
               'hc_usdt', 'node_usdt', 'one_usdt', 'let_usdt', 'sun_usdt', 'ruff_usdt', 'pols_usdt', 'nkn_usdt',
               'woo_usdt', 'storj_usdt', 'blz_usdt', 'comp_usdt', 'axs_usdt', 'akro_usdt', 'pvt_usdt', 'ogo_usdt',
               'smt_usdt', 'waxp_usdt', 'avax_usdt', 'cmt_usdt', 'waves_usdt', 'inj_usdt', 'top_usdt', 'swftc_usdt',
               'ant_usdt', 'ankr_usdt', 'lamb_usdt', 'bts_usdt', 'zil_usdt', 'tt_usdt', 'zrx_usdt', 'ocn_usdt',
               'luna_usdt', 'badger_usdt', 'nas_usdt', 'bhd_usdt', 'atp_usdt', 'rsr_usdt', 'vsys_usdt', 'mds_usdt',
               'trb_usdt', 'iris_usdt', 'em_usdt', 'for_usdt', 'cro_usdt', 'fis_usdt', 'ela_usdt', 'snt_usdt',
               'vidy_usdt', 'elf_usdt', 'uma_usdt', 'rvn_usdt', 'pond_usdt', 'dock_usdt', 'mkr_usdt', 'beth_usdt',
               'sand_usdt', 'ctxc_usdt', 'ogn_usdt', 'cre_usdt', 'ftt_usdt', 'gnx_usdt', 'kava_usdt', 'df_usdt',
               'stpt_usdt', 'cvc_usdt', 'wnxm_usdt', 'icx_usdt', 'oxt_usdt', 'nbs_usdt', 'lba_usdt', 'firo_usdt',
               'band_usdt', 'wtc_usdt', 'dcr_usdt', 'steem_usdt', 'abt_usdt', 'knc_usdt', 'nexo_usdt', 'utk_usdt',
               'hive_usdt', 'loom_usdt', 'arpa_usdt', 'hbc_usdt', 'dai_usdt', 'mln_usdt', 'nuls_usdt', 'bal_usdt',
               'ast_usdt', 'bnt_usdt', 'glm_usdt', 'pha_usdt', 'bags_usdt', 'skl_usdt', 'nu_usdt', 'bsv3l_usdt',
               'bsv3s_usdt', 'eos3l_usdt', 'eos3s_usdt', 'dot2l_usdt', 'dot2s_usdt', 'uni2l_usdt', 'uni2s_usdt',
               'ltc3l_usdt', 'ltc3s_usdt', 'xrp3l_usdt', 'xrp3s_usdt', 'bch3l_usdt', 'bch3s_usdt', 'zec3l_usdt',
               'zec3s_usdt', 'fil3l_usdt', 'fil3s_usdt', 'eth1s_usdt', 'btc1s_usdt', 'o3_usdt', 'rlc_usdt', 'apn_usdt',
               'lhb_usdt', 'titan_usdt', 'rndr_usdt', 'nhbtc_usdt', 'yam_usdt', 'cru_usdt', 'reef_usdt', 'value_usdt',
               'dht_usdt', 'filda_usdt', 'nsure_usdt', 'ctsi_usdt', 'insur_usdt', 'stn_usdt', 'gof_usdt', 'can_usdt',
               'lina_usdt', 'rai_usdt', 'stake_usdt', 'mir_usdt', 'forth_usdt', 'swrv_usdt', 'xrt_usdt', 'auction_usdt',
               'bcha_usdt', 'mta_usdt', 'pearl_usdt', 'dka_usdt', 'front_usdt', 'cvp_usdt', 'api3_usdt', 'bor_usdt',
               'fti_usdt', 'dac_usdt', 'ekt_usdt', 'uuu_usdt', 'xmx_usdt', 'aac_usdt', 'uip_usdt', 'seele_usdt',
               'lxt_usdt', 'ae_usdt', 'mxc_usdt', 'ren_usdt', 'kcash_usdt', 'hot_usdt']

In [205]:
symbols_klinedata=get_all_symbols_klinedata('huobi')

In [208]:
j=0
for i in symbols_klinedata:
    symbol=i[0]
    klinedata=i[1]
    if klinedata:
        #print(symbol)
        j+=1
        print(j)
    #print(symbol,gold_cross(symbol,klinedata))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197


In [209]:
MA_data=MA(klinedata)

In [210]:
MA_data

,time,low,close,volumn,MA5,MA30,MA_sign
0,2021-05-10 02:00:00,0.018326,0.018403,4.146942e+05,NaN,NaN,0
1,2021-05-10 03:00:00,0.018313,0.018481,6.697356e+05,NaN,NaN,0
2,2021-05-10 04:00:00,0.018419,0.018711,3.465708e+05,NaN,NaN,0
3,2021-05-10 05:00:00,0.018491,0.018839,6.804230e+05,NaN,NaN,0
4,2021-05-10 06:00:00,0.018487,0.018658,6.857302e+05,0.018618,NaN,0
...,...,...,...,...,...,...,...
195,2021-05-18 05:00:00,0.013677,0.013841,8.496141e+05,0.014104,0.014394,0
196,2021-05-18 06:00:00,0.013680,0.013797,3.420018e+05,0.014084,0.014362,0
197,2021-05-18 07:00:00,0.013746,0.013996,1.297748e+06,0.014059,0.014335,0
198,2021-05-18 08:00:00,0.013913,0.014171,1.147472e+06,0.014053,0.014331,0


In [253]:
import datetime
now = datetime.datetime.now()
def gold_cross(symbol,klinedata):
    flag = 0
    if not klinedata:
        return flag
    MA_data = MA(klinedata)
    for i in range(190):
        flag = 0
        ma_sign_array = MA_data['MA_sign'].values.tolist()[:-i]
        low_array = MA_data['low'].values.tolist()[:-i]
        volumn_array = MA_data['volumn'].values.tolist()[:-i]
        try:
            if ma_sign_array[-1] == 1 and volumn_array:
                dead_cross_position = ma_sign_array[::-1].index(2)
                low_array = low_array[-(dead_cross_position + 1):]
                volumn_array = volumn_array[-(dead_cross_position + 1):]
                low_min_position = low_array.index(min(low_array))
                if volumn_array[:low_min_position]:
                    volumn_mean = np.mean(volumn_array[:low_min_position])
                    if volumn_array[low_min_position] >= volumn_mean * 2.5:
                        flag = 1
                        day_delta=i//24
                        hour=i%24
                        if hour<=now.hour:
                            cross_date = datetime.datetime(now.year,now.month,now.day-day_delta,now.hour-hour)
                        else:
                            hour=hour-now.hour
                            cross_date = datetime.datetime(now.year,now.month,now.day-day_delta-1,24-hour)
                        #print("交易对{}在{}处出现买入信号{}".format(symbol,cross_date,flag))
                        print("{}-{}".format(symbol,cross_date))
        except Exception as e:
            pass
            

In [254]:
for i in symbols_klinedata:
    symbol = i[0]
    klinedata = i[1]
    gold_cross(symbol,klinedata)

dot_usdt-2021-05-13 15:00:00
bsv_usdt-2021-05-14 11:00:00
eos_usdt-2021-05-16 15:00:00
usdc_usdt-2021-05-13 13:00:00
nano_usdt-2021-05-13 07:00:00
chz_usdt-2021-05-15 09:00:00
atom_usdt-2021-05-14 12:00:00
theta_usdt-2021-05-17 17:00:00
theta_usdt-2021-05-14 09:00:00
lol_usdt-2021-05-16 17:00:00
lol_usdt-2021-05-14 16:00:00
iost_usdt-2021-05-14 10:00:00
vet_usdt-2021-05-14 09:00:00
tnb_usdt-2021-05-16 12:00:00
tnb_usdt-2021-05-15 20:00:00
dta_usdt-2021-05-16 15:00:00
neo_usdt-2021-05-14 11:00:00
jst_usdt-2021-05-14 10:00:00
xem_usdt-2021-05-14 10:00:00
lrc_usdt-2021-05-16 18:00:00
lrc_usdt-2021-05-15 11:00:00
mask_usdt-2021-05-14 05:00:00
sol_usdt-2021-05-14 18:00:00
bat_usdt-2021-05-14 11:00:00
itc_usdt-2021-05-18 07:00:00
bix_usdt-2021-05-16 17:00:00
sc_usdt-2021-05-14 11:00:00
mass_usdt-2021-05-14 09:00:00
fsn_usdt-2021-05-13 19:00:00
algo_usdt-2021-05-14 00:00:00
xtz_usdt-2021-05-14 11:00:00
ring_usdt-2021-05-14 08:00:00
node_usdt-2021-05-16 18:00:00
one_usdt-2021-05-14 09:00:00
nk

In [232]:
import datetime
a = datetime.datetime.now()
b = datetime.datetime(a.year,a.month,a.day,a.hour-8,a.minute,a.second,a.microsecond)
print(b)

2021-05-18 01:20:05.136777


In [229]:
i=53
day=i//24
print(day)

2


In [230]:
hour=i%24
print(hour)

5
